In [1]:
import pandas as pd
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]


In [2]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    titulo = nome.split(',')[1].split('.')[0].strip()
    # if (titulo in ['Mille','Ms','Lady', 'Miss']):
    #     titulo = 'Miss'
    # elif (titulo in ['Mme', 'Sir']):
    #     titulo = 'Mrs'
    # elif (titulo in ['Master', 'Mr', 'Mrs']):
    #     titulo = 'Other'
    return titulo

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
        return Xdrop


In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()



In [5]:
class AtributoIdade(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.idade = 1
        return self
    def transform(self, X, y=None):
         idades = X[:, self.idade]
         cat_idades = pd.cut(idades,[0,9,18,30,40,50,100], labels=[9,18,30,40,50,100]).astype(int)
         X[:, self.idade] = cat_idades
         return X

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('idade', AtributoIdade()),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [354]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier(n_jobs=-1))
])

parametros = {
    'atributosDesejados__excluirName': [True],
    'classificador__max_depth': [5],
    'classificador__n_estimators': [10, 25, 50, 100], 

}
modelo = GridSearchCV(pipetotal, param_grid=parametros, n_jobs=-1)

scores = cross_validate(modelo, X, y, cv=RepeatedKFold(n_repeats=15, n_splits=10))
np.mean(scores['test_score']), np.std(scores['test_score'])

In [340]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)